### <center>2018 Winter CS101.07</center>

# <center>链表</center>

##### <center>by tanzhuxiaqiu@huawei.com</center>

## 回顾

- 动态数组
- 栈
- 队列

## 今日议程

1. 单向链表
2. 环形链表
3. 双向链表
4. 位置链表
5. 链表 v.s. 数组

## 单向链表

多个节点（Node）集合组成的一个线性序列。

#### 单向链表的节点

- 元素成员：表示节点存储的元素值。
- 指针成员：指向单向链表中的后继节点，如果没有后继则为空。

![](img/7-1.png)

- 头节点：链表中第一个节点
- 尾节点：链表中最后一个节点

![](img/7-2.png)

In [4]:
class SinglyLinkedListNode:

    def __init__(self, e):
        self._element = e
        self._next = None

### 单向链表的几个基本操作


[visualgo](https://visualgo.net/en/list)

#### 在单向链表头部插入一个元素

伪代码
```
Algorithm add_first(L, e):
    n = Node(e)
    n.next = L.head
    L.head = n
    L.size += 1
```

#### 在单向链表尾部插入一个元素

伪代码

```
Algorithm add_last(L, e):
    n = Node(e)
    n.next = None
    L.tail.next = n
    L.tail = n
    L.size += 1
```

#### 在单向链表头部删除一个元素

伪代码

```
Algorithm del_first(L):
    if L.head in None then
        raise an error: the list is empty
    L.head = L.head.next
    L.size -= 1
```

#### 思考：如何在单向链表的尾部删除一个元素？

### 利用单向链表实现一个栈

- 把单向链表的头部作为栈顶

In [5]:
from abc import ABC, abstractmethod


class AbstractStack(ABC):
    """Abstract Class for Stacks."""
    def __init__(self):
        self._top = -1

    def __len__(self):
        return self._top + 1

    def __repr__(self):
        res = "->".join(map(str, self))
        return 'Top->' + res

    def is_empty(self):
        return self._top == -1

    @abstractmethod
    def __iter__(self):
        raise NotImplementedError

    @abstractmethod
    def push(self, e):
        raise NotImplementedError

    @abstractmethod
    def pop(self):
        raise NotImplementedError

    @abstractmethod
    def peek(self):
        raise NotImplementedError


In [6]:
class LinkedListStack(AbstractStack):

    def __init__(self):
        super().__init__()
        self._head = None

    def __iter__(self):
        p = self._head
        while p is not None:
            yield p._element
            p = p._next

    def push(self, e):
        node = SinglyLinkedListNode(e)
        node._next = self._head
        self._head = node
        self._top += 1

    def pop(self):
        if self.is_empty():
            raise IndexError('Stack is empty')
        res = self._head._element
        self._head = self._head._next
        self._top -= 1
        return res

    def peek(self):
        if self.is_empty():
            raise IndexError('Stack is empty')
        return self._head._element

In [7]:
stack = LinkedListStack()
for i in range(5):
    stack.push(i)
stack

Top->4->3->2->1->0

In [8]:
print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack)

4
3
2
Top->1->0


#### 基于单向链表的栈的时间复杂度

|操作|示例|时间复杂度|
|---|---|---|---|
| Push | S.push(e) | O(1) |
| Pop | S.pop() | O(1) |
| Peek | S.peek() | O(1) |
| Is empty | S.is_empty() | O(1) |
| Length | len(S) | O(1) |

### 利用单向链表实现一个队列

- 需要维护头部和尾部两个指针
- 因为很难在单向链表尾部删除元素，所以出队列的操作应该放在单向链表的头部完成
- 入队在单向链表的尾部完成

In [9]:
from abc import ABC, abstractmethod


class AbstractQueue(ABC):

    def __init__(self):
        self._size = 0

    def __len__(self):
        return self._size

    def __repr__(self):
        return "->".join(map(str, self))

    def is_empty(self):
        return self._size == 0

    @abstractmethod
    def enqueue(self, e):
        raise NotImplementedError

    @abstractmethod
    def dequeue(self):
        raise NotImplementedError

    @abstractmethod
    def peek(self):
        raise NotImplementedError

    @abstractmethod
    def __iter__(self):
        raise NotImplementedError

In [13]:
class LinkedListQueue(AbstractQueue):

    def __init__(self):
        super().__init__()
        self._head = None
        self._tail = None

    def __iter__(self):
        p = self._head
        while p is not None:
            yield p._element
            p = p._next

    def enqueue(self, e):
        node = SinglyLinkedListNode(e)
        if self.is_empty():
            self._head = node
        else:
            self._tail._next = node
        self._tail = node
        self._size += 1

    def dequeue(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        res = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():
            self._tail = None
        return res

    def peek(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        return self._head._element

In [14]:
queue = LinkedListQueue()
for i in range(5):
    queue.enqueue(i)
print(queue)

0->1->2->3->4


In [15]:
print(queue.dequeue())
print(queue.dequeue())
print(queue.dequeue())
print(queue)

0
1
2
3->4


## 环形链表

### Circular Linked List

- 将链表尾部的指针指向链表自己的头部
- 相比利用数组和取模运算实现的环形结构，环形链表实现了物理上的闭环、

![](img/7-3.png)

### 利用环形链表实现环形队列

- 只需要一个尾指针即可完成队列的基本操作
- 新增一个rotate()方法实现队列的轮转（头部第一个元素排到队列的末尾，也可以理解为第一个元素出列后再入列）

In [17]:
class CircularQueue(AbstractQueue):
    def __init__(self):
        super().__init__()
        self._tail = None

    def __iter__(self):
        p = self._tail._next
        while True:
            yield p._element
            if p is self._tail:
                return
            p = p._next

    def enqueue(self, e):
        node = SinglyLinkedListNode(e)
        if self.is_empty():
            node._next = node
        else:
            node._next = self._tail._next
            self._tail._next = node
        self._tail = node
        self._size += 1

    def dequeue(self):
        remove_node = self._tail._next
        if len(self) == 1:
            self._tail = None
        else:
            self._tail._next = remove_node._next
        self._size -= 1
        return remove_node._element

    def peek(self):
        return self._tail._next._element

    def rotate(self):
        if len(self) > 0:
            self._tail = self._tail._next

In [18]:
queue = CircularQueue()
for i in range(5):
    queue.enqueue(i)
print(queue)
print(queue.dequeue())
print(queue.dequeue())

0->1->2->3->4
0
1


In [20]:
print(queue)
queue.rotate()
print(queue)

2->3->4
3->4->2


## 双向链表

因为单向链表的不对称性带来了一些缺陷：

- 难以删除链表中除头节点外的任意节点